# Preprocesses log data and create a pickle file for further analysis

In [47]:
import sys
import os
modules_path = os.getcwd() + '/..'
sys.path.insert(0, modules_path)

# imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import math
from notebooks.utils import compute_user_penalty, get_team_values_df
from common.load import load_competition_data, process_team_logs

pd.set_option('display.max_colwidth', None)
unknownRankLimit = 1000
unknownRankValue = 2000

# Import common data

In [48]:
config = modules_path +'/config_vbs2023.yaml'

# load competition data from dres files and auxiliary data (FPSs, sequences)
comp_data = load_competition_data(config)

# compute team logs and put them in the form of dataframes (or load them if already existing)
logs = process_team_logs(config, comp_data, force=False)

teams = comp_data['config']['teams']
teams.remove('VideoCLIP')

# FIXME: badly patch the name of HTW into vibro
teams_adj = teams.copy()
teams_adj = [t if t != 'HTW' else 'vibro' for t in teams_adj]

# valid teams
team_order = ['vibro', 'VISIONE', 'vitrivr-VR', 'CVHunter',  'Verge']
#team_order = ['vibro', 'VISIONE', 'VIREO' 'vitrivr-VR', 'CVHunter', 'vitrivr', 'Verge'] // full list

c:\Lang\Python\VBS23-Post-Hoc-Analysis\src\python\notebooks/..\common\tasks.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.tasks_df = self.tasks_df.append({
c:\Lang\Python\VBS23-Post-Hoc-Analysis\src\python\notebooks/..\common\tasks.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.tasks_df = self.tasks_df.append({
c:\Lang\Python\VBS23-Post-Hoc-Analysis\src\python\notebooks/..\common\tasks.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.tasks_df = self.tasks_df.append({
c:\Lang\Python\VBS23-Post-Hoc-Analysis\src\python\notebooks/..\common\tasks.py:59: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  self.tasks_

# Merge team data into one dataframe

In [49]:
teams

['HTW', 'VISIONE', 'VIREO', 'vitrivr-VR', 'CVHunter', 'vitrivr', 'Verge']

In [50]:
dfs = []
for team in teams:
    team_df = logs[team].get_events_dataframe().reset_index()
    dfs.append(team_df)

dataset = pd.concat(dfs, axis=0).reset_index()
dataset.head()

,level_0,index,task,team,user,timestamp,elapsed_since_task_start_ms,correct_submission_time_ms,rank_video,rank_shot_margin_0,rank_shot_margin_5,max_rank,category,type,value,additionals,is_user_with_correct_submission
0,0,0,vbs23-kis-v1,HTW,0,1673259650000,36687,43981.0,1.0,1.0,1.0,10000.0,TEXT,jointEmbedding,Man in pink closest standing a basketball stadium,{},NaN
1,1,1,vbs23-kis-t1,HTW,0,1673260401000,28139,36751.0,1.0,1.0,1.0,10000.0,TEXT,jointEmbedding,"Kids in kayaks on a river, throwing paddles through three coloured hoops placed over the water.",{},NaN
2,2,2,vbs23-kis-t2,HTW,0,1673260903000,6356,26109.0,1.0,1.0,1.0,10000.0,TEXT,jointEmbedding,Hands of a kid applying glue to an egg carton and then a view of a sculpture made of those cartons.,{},NaN
3,3,3,vbs23-kis-t3,HTW,0,1673261414000,12701,394938.0,670.0,670.0,670.0,10000.0,TEXT,jointEmbedding,A view through the glass roof of a lift cabin while it is moving upwards.,{},NaN
4,4,4,vbs23-kis-t3,HTW,0,1673261434000,32701,394938.0,75.0,75.0,75.0,10000.0,IMAGE,globalFeatures,video 14479 at 00:08:18.500 (Frame #12462),{},NaN


# Data checks
Displaying special log information that concerns only a few teams.

### all category and type combination per team

In [51]:
print('\n'.join(map(str, dataset[["team","category","type"]].groupby(["team","category","type"]).groups.keys())))

('CVHunter', 'BROWSING', '3P/Video')
('CVHunter', 'BROWSING', 'DetailWindowClose')
('CVHunter', 'BROWSING', 'DetailWindowOpen')
('CVHunter', 'BROWSING', 'SimilarityDetailClose')
('CVHunter', 'BROWSING', 'TextSuggestionSelected')
('CVHunter', 'BROWSING', 'VideoDetailClose')
('CVHunter', 'BROWSING', 'VideoDetailOpen')
('CVHunter', 'IMAGE', 'JointEmbedding')
('CVHunter', 'IMAGE', 'feedbackModel')
('CVHunter', 'SUBMIT', nan)
('CVHunter', 'TEXT', 'JointEmbedding')
('HTW', 'IMAGE', 'globalFeatures')
('HTW', 'TEMPORAL', 'text > text')
('HTW', 'TEXT', 'jointEmbedding')
('VIREO', nan, nan)
('VISIONE', 'IMAGE', 'globalFeatures > image similarity (GEM)')
('VISIONE', 'IMAGE', 'globalFeatures > sematic video similarity (CLIP)')
('VISIONE', 'TEXT', 'LocalizedObjectAndColors > classes')
('VISIONE', 'TEXT', 'LocalizedObjectAndColors > position')
('VISIONE', 'TEXT', 'jointEmbedding > ')
('Verge', 'FILTER', 'numberOfObjects')
('Verge', 'IMAGE', 'globalFeatures')
('Verge', 'TEXT', 'concept')
('Verge', 'T

### only one sketch query

In [52]:
dataset.loc[dataset.category == "SKETCH"]

,level_0,index,task,team,user,timestamp,elapsed_since_task_start_ms,correct_submission_time_ms,rank_video,rank_shot_margin_0,rank_shot_margin_5,max_rank,category,type,value,additionals,is_user_with_correct_submission
1409,44,44,vbs23-kis-v-m4,vitrivr,0,1673272491103,117918,NaN,8649.0,8649.0,8649.0,10000.0,SKETCH,NaN,color,{},NaN


### only one filter query

In [53]:
dataset.loc[dataset.category == "FILTER"]

,level_0,index,task,team,user,timestamp,elapsed_since_task_start_ms,correct_submission_time_ms,rank_video,rank_shot_margin_0,rank_shot_margin_5,max_rank,category,type,value,additionals,is_user_with_correct_submission
1589,105,105,vbs23-kis-t6,Verge,0,1673264294471,131097,179991.0,3.0,3.0,3.0,232.0,FILTER,numberOfObjects,2,{},NaN


### ### only one visualtextcoembedding query

In [54]:
dataset.loc[dataset.category == "visualtextcoembedding"]

,level_0,index,task,team,user,timestamp,elapsed_since_task_start_ms,correct_submission_time_ms,rank_video,rank_shot_margin_0,rank_shot_margin_5,max_rank,category,type,value,additionals,is_user_with_correct_submission
644,1,1,vbs23-kis-v1,vitrivr-VR,0,1673259676512,68199,171058.0,4441.0,inf,inf,NaN,visualtextcoembedding,TEXT,man in a sports hall doing a backflip,NaN,NaN


### vitrivr's value column is empty

In [55]:
dataset.loc[dataset.team == "vitrivr"]

,level_0,index,task,team,user,timestamp,elapsed_since_task_start_ms,correct_submission_time_ms,rank_video,rank_shot_margin_0,rank_shot_margin_5,max_rank,category,type,value,additionals,is_user_with_correct_submission
1365,0,0,vbs23-kis-v1,vitrivr,0,1673259630775,17462,68132.0,inf,inf,inf,4990.0,TEXT,NaN,NaN,{},NaN
1366,1,1,vbs23-kis-t1,vitrivr,0,1673260395711,22850,93575.0,3.0,4.0,4.0,4981.0,TEXT,NaN,NaN,{},NaN
1367,2,2,vbs23-kis-t1,vitrivr,0,1673260461824,88963,93575.0,10.0,10.0,10.0,4988.0,TEXT,NaN,NaN,{},NaN
1368,3,3,vbs23-kis-t2,vitrivr,0,1673260917544,20900,218530.0,200.0,200.0,200.0,4991.0,TEXT,NaN,NaN,{},NaN
1369,4,4,vbs23-kis-t2,vitrivr,0,1673260989249,92605,218530.0,71.0,71.0,71.0,4991.0,TEXT,NaN,NaN,{},NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1479,114,114,vbs23-kis-v5,vitrivr,1,1673278598476,52235,NaN,242.0,2816.0,2816.0,9950.0,TEXT,NaN,NaN,{},NaN
1480,115,115,vbs23-kis-v5,vitrivr,1,1673278614088,67847,NaN,242.0,2816.0,2816.0,9950.0,TEXT,NaN,NaN,{},NaN
1481,116,116,vbs23-kis-v5,vitrivr,1,1673278671514,125273,NaN,40.0,1167.0,1167.0,4981.0,TEXT,NaN,NaN,{},NaN
1482,117,117,vbs23-kis-v5,vitrivr,1,1673278736822,190581,NaN,23.0,739.0,739.0,4986.0,TEXT,NaN,NaN,{},NaN


### VIREO's value column is empty

In [56]:
dataset.loc[dataset.team == "VIREO"]

,level_0,index,task,team,user,timestamp,elapsed_since_task_start_ms,correct_submission_time_ms,rank_video,rank_shot_margin_0,rank_shot_margin_5,max_rank,category,type,value,additionals,is_user_with_correct_submission
407,0,0,vbs23-kis-v1,VIREO,0,1673259634037,20724,48547.0,794.0,inf,inf,1000.0,NaN,NaN,NaN,NaN,NaN
408,1,1,vbs23-kis-v1,VIREO,0,1673259636735,23422,48547.0,300.0,inf,inf,1000.0,NaN,NaN,NaN,NaN,NaN
409,2,2,vbs23-kis-v1,VIREO,0,1673259649218,35905,48547.0,260.0,inf,inf,1000.0,NaN,NaN,NaN,NaN,NaN
410,3,3,vbs23-kis-t1,VIREO,0,1673260405338,32477,166258.0,inf,inf,inf,1000.0,NaN,NaN,NaN,NaN,NaN
411,4,4,vbs23-kis-t1,VIREO,0,1673260449743,76882,166258.0,inf,inf,inf,1000.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
638,231,231,vbs23-kis-v4,VIREO,1,1673277200248,23035,45771.0,163.0,inf,inf,1000.0,NaN,NaN,NaN,NaN,NaN
639,232,232,vbs23-kis-v4,VIREO,1,1673277208737,31524,45771.0,141.0,141.0,141.0,1000.0,NaN,NaN,NaN,NaN,NaN
640,233,233,vbs23-kis-t7,VIREO,1,1673277558871,27337,191328.0,6.0,inf,6.0,1000.0,NaN,NaN,NaN,NaN,NaN
641,234,234,vbs23-kis-t7,VIREO,1,1673277697834,166300,191328.0,5.0,inf,5.0,1000.0,NaN,NaN,NaN,NaN,NaN


### vitrivr-VR types and category columns are swapped

In [57]:
dataset.loc[dataset.team == "vitrivr-VR"].head()

,level_0,index,task,team,user,timestamp,elapsed_since_task_start_ms,correct_submission_time_ms,rank_video,rank_shot_margin_0,rank_shot_margin_5,max_rank,category,type,value,additionals,is_user_with_correct_submission
643,0,0,vbs23-kis-v1,vitrivr-VR,0,1673259638599,30286,171058.0,inf,inf,inf,NaN,mlclip,TEXT,man in a sports hall doing a backflip,NaN,NaN
644,1,1,vbs23-kis-v1,vitrivr-VR,0,1673259676512,68199,171058.0,4441.0,inf,inf,NaN,visualtextcoembedding,TEXT,man in a sports hall doing a backflip,NaN,NaN
645,2,2,vbs23-kis-v1,vitrivr-VR,1,1673259678971,70658,171058.0,inf,inf,inf,NaN,mlclip,TEXT,man in blue shirt doing backflip,NaN,NaN
646,3,3,vbs23-kis-v1,vitrivr-VR,0,1673259716531,108218,171058.0,74.0,834.0,834.0,NaN,mlclip,TEXT,man wearing american flag in a sports hall doing a backflip,NaN,NaN
647,4,4,vbs23-kis-v1,vitrivr-VR,0,1673259745302,136989,171058.0,62.0,189.0,189.0,NaN,mlclip,TEXT,a man in a white shirt and blue pants dancing in a sports hall,NaN,NaN


### verge concept type query has no values

In [58]:
dataset.loc[((dataset.team == "Verge") & (dataset.type=="concept"))].head()

,level_0,index,task,team,user,timestamp,elapsed_since_task_start_ms,correct_submission_time_ms,rank_video,rank_shot_margin_0,rank_shot_margin_5,max_rank,category,type,value,additionals,is_user_with_correct_submission
1496,12,12,vbs23-kis-v1,Verge,0,1673259819531,206218,221930.0,inf,inf,inf,2691.0,TEXT,concept,",,",{},NaN
1507,23,23,vbs23-kis-t1,Verge,0,1673260586873,214012,281491.0,inf,inf,inf,10000.0,TEXT,concept,NaN,{},NaN
1509,25,25,vbs23-kis-t1,Verge,0,1673260597146,224285,281491.0,inf,inf,inf,262.0,TEXT,concept,",",{},NaN
1527,43,43,vbs23-kis-t2,Verge,0,1673261206262,309618,NaN,inf,inf,inf,1174.0,TEXT,concept,",",{},NaN
1580,96,96,vbs23-kis-t5,Verge,0,1673263133819,307207,NaN,8534.0,inf,9109.0,10000.0,TEXT,concept,NaN,{},NaN


### HTW temporal category has only text queries

In [59]:
dataset.loc[dataset.category=="TEMPORAL"].head()

,level_0,index,task,team,user,timestamp,elapsed_since_task_start_ms,correct_submission_time_ms,rank_video,rank_shot_margin_0,rank_shot_margin_5,max_rank,category,type,value,additionals,is_user_with_correct_submission
42,42,42,vbs23-kis-v-m4,HTW,0,1673272547000,173815,211429.0,21.0,21.0,21.0,10000.0,TEMPORAL,text > text,Corals a lot of fish in blue water. > Blue water and corals,{},NaN
43,43,43,vbs23-kis-v-m4,HTW,0,1673272551000,177815,211429.0,17.0,17.0,17.0,10000.0,TEMPORAL,text > text,Corals a lot of fish in blue water. > Blue water and corals,{},NaN
44,44,44,vbs23-kis-v-m4,HTW,0,1673272583000,209815,211429.0,12.0,12.0,12.0,10000.0,TEMPORAL,text > text,Corals a lot of fish in blue water. > Blue water and corals,{},NaN
120,120,120,vbs23-kis-v-m4,HTW,1,1673272504000,130815,211429.0,636.0,893.0,893.0,10000.0,TEMPORAL,text > text,manz fishes over red corals > red corals,{},NaN
121,121,121,vbs23-kis-v-m4,HTW,1,1673272520000,146815,211429.0,717.0,945.0,945.0,10000.0,TEMPORAL,text > text,manz fishes over red corals > red corals,{},NaN


### CV-Hunter has a CLIP: or Temporal-CLIP: prefix in the value column for text queries

In [60]:
dataset.loc[dataset.team=="CVHunter"].head()

,level_0,index,task,team,user,timestamp,elapsed_since_task_start_ms,correct_submission_time_ms,rank_video,rank_shot_margin_0,rank_shot_margin_5,max_rank,category,type,value,additionals,is_user_with_correct_submission
704,0,0,vbs23-kis-v1,CVHunter,1,1673259626784,13471,NaN,inf,inf,inf,NaN,TEXT,JointEmbedding,"CLIP: man in sport hall, backflip",NaN,0.0
705,1,1,vbs23-kis-v1,CVHunter,1,1673259628298,14985,NaN,inf,inf,inf,NaN,TEXT,JointEmbedding,"CLIP: man in sport hall, backflip",NaN,0.0
706,2,2,vbs23-kis-v1,CVHunter,0,1673259628480,15167,NaN,inf,inf,inf,NaN,TEXT,JointEmbedding,CLIP: man making salto on a stadion,NaN,0.0
707,3,3,vbs23-kis-v1,CVHunter,0,1673259630309,16996,NaN,inf,inf,inf,NaN,TEXT,JointEmbedding,CLIP: man making salto on a stadion,NaN,0.0
708,4,4,vbs23-kis-v1,CVHunter,1,1673259634686,21373,NaN,inf,inf,inf,NaN,TEXT,JointEmbedding,"CLIP: man in sport hall, backflip, american",NaN,0.0


### CVHunter has some browsing category entries but they do not change the ranking order

In [61]:
print(dataset.loc[((dataset.team == "CVHunter") & (dataset.category=="BROWSING"))].type.unique())
dataset.loc[((dataset.team == "CVHunter") & (dataset.category=="BROWSING"))]

['SimilarityDetailClose' 'DetailWindowOpen' 'DetailWindowClose'
 'VideoDetailOpen' 'VideoDetailClose' 'TextSuggestionSelected' '3P/Video']


,level_0,index,task,team,user,timestamp,elapsed_since_task_start_ms,correct_submission_time_ms,rank_video,rank_shot_margin_0,rank_shot_margin_5,max_rank,category,type,value,additionals,is_user_with_correct_submission
721,17,17,vbs23-kis-v1,CVHunter,1,1673259682958,69645,NaN,inf,inf,inf,NaN,BROWSING,SimilarityDetailClose,14556:4540,1673259667464,0.0
729,25,25,vbs23-kis-v1,CVHunter,1,1673259716262,102949,NaN,inf,inf,inf,NaN,BROWSING,SimilarityDetailClose,15907:150509,1673259667464,0.0
731,27,27,vbs23-kis-v1,CVHunter,1,1673259728930,115617,NaN,inf,inf,inf,NaN,BROWSING,SimilarityDetailClose,13583:1513,1673259667464,0.0
743,39,39,vbs23-kis-v1,CVHunter,1,1673259829428,216115,NaN,inf,inf,inf,NaN,BROWSING,SimilarityDetailClose,5494:41100,1673259761185,0.0
755,51,51,vbs23-kis-t1,CVHunter,0,1673260402970,30109,345478.0,inf,inf,inf,NaN,BROWSING,DetailWindowOpen,15022:6054,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1351,647,647,vbs23-kis-v-m4,CVHunter,1,1673272665824,292639,NaN,inf,inf,inf,NaN,BROWSING,VideoDetailOpen,601:420,NaN,0.0
1352,648,648,vbs23-kis-v-m4,CVHunter,1,1673272670289,297104,NaN,inf,inf,inf,NaN,BROWSING,DetailWindowOpen,601:142,NaN,0.0
1353,649,649,vbs23-kis-v-m4,CVHunter,1,1673272672143,298958,NaN,inf,inf,inf,NaN,BROWSING,DetailWindowClose,601:142,NaN,0.0
1358,654,654,vbs23-kis-v-m5,CVHunter,1,1673272787905,34615,38507.0,inf,inf,inf,NaN,BROWSING,DetailWindowOpen,295:225,NaN,0.0


### CVHunter has submit entries which are not important for the ranking or text analysis

In [62]:
dataset.loc[((dataset.team == "CVHunter") & (dataset.category=="SUBMIT"))].head()

,level_0,index,task,team,user,timestamp,elapsed_since_task_start_ms,correct_submission_time_ms,rank_video,rank_shot_margin_0,rank_shot_margin_5,max_rank,category,type,value,additionals,is_user_with_correct_submission
805,101,101,vbs23-kis-t1,CVHunter,1,1673260718364,345503,345478.0,inf,inf,inf,NaN,SUBMIT,NaN,?item=13872&frame=1481&session=node0ncp3v4rmms401wddfckpfb4w7912,"{""submission"":""CORRECT"" ""description"":""Submission correct!"" ""status"":true}",1.0
825,121,121,vbs23-kis-t2,CVHunter,1,1673260999671,103027,102992.0,inf,inf,inf,NaN,SUBMIT,NaN,?item=13062&frame=3650&session=node0ncp3v4rmms401wddfckpfb4w7912,"{""submission"":""CORRECT"" ""description"":""Submission correct!"" ""status"":true}",1.0
1032,328,328,vbs23-kis-t5,CVHunter,0,1673262932130,105518,105663.0,inf,inf,inf,NaN,SUBMIT,NaN,?item=05722&frame=5025&session=node01f46iddshud8duses24frm8g927,"{""submission"":""CORRECT"" ""description"":""Submission correct!"" ""status"":true}",1.0
1057,353,353,vbs23-kis-t6,CVHunter,0,1673264305097,141723,141909.0,inf,inf,inf,NaN,SUBMIT,NaN,?item=12222&frame=656&session=node01f46iddshud8duses24frm8g927,"{""submission"":""CORRECT"" ""description"":""Submission correct!"" ""status"":true}",1.0
1118,414,414,vbs23-kis-v3,CVHunter,0,1673276456694,49823,50227.0,inf,inf,inf,NaN,SUBMIT,NaN,?item=16276&frame=444&session=node0mq1oq40r6jl01v9ijyl85bqtx1254,"{""submission"":""CORRECT"" ""description"":""Submission correct!"" ""status"":true}",1.0


### CVHunter has triggert every text query two times

In [63]:
dataset.loc[((dataset.team == "CVHunter") & (dataset.category=="TEXT"))].head()

,level_0,index,task,team,user,timestamp,elapsed_since_task_start_ms,correct_submission_time_ms,rank_video,rank_shot_margin_0,rank_shot_margin_5,max_rank,category,type,value,additionals,is_user_with_correct_submission
704,0,0,vbs23-kis-v1,CVHunter,1,1673259626784,13471,NaN,inf,inf,inf,NaN,TEXT,JointEmbedding,"CLIP: man in sport hall, backflip",NaN,0.0
705,1,1,vbs23-kis-v1,CVHunter,1,1673259628298,14985,NaN,inf,inf,inf,NaN,TEXT,JointEmbedding,"CLIP: man in sport hall, backflip",NaN,0.0
706,2,2,vbs23-kis-v1,CVHunter,0,1673259628480,15167,NaN,inf,inf,inf,NaN,TEXT,JointEmbedding,CLIP: man making salto on a stadion,NaN,0.0
707,3,3,vbs23-kis-v1,CVHunter,0,1673259630309,16996,NaN,inf,inf,inf,NaN,TEXT,JointEmbedding,CLIP: man making salto on a stadion,NaN,0.0
708,4,4,vbs23-kis-v1,CVHunter,1,1673259634686,21373,NaN,inf,inf,inf,NaN,TEXT,JointEmbedding,"CLIP: man in sport hall, backflip, american",NaN,0.0


### VISIONE has two LocalizedObjectAndColor queries

In [64]:
dataset.loc[((dataset.team == "VISIONE") & (dataset.type.str.contains('LocalizedObjectAndColor')))].head()

,level_0,index,task,team,user,timestamp,elapsed_since_task_start_ms,correct_submission_time_ms,rank_video,rank_shot_margin_0,rank_shot_margin_5,max_rank,category,type,value,additionals,is_user_with_correct_submission
345,178,178,vbs23-kis-v-m5,VISIONE,1,1673272765946,12656,51113.0,1780.0,2143.0,1780.0,9928.0,TEXT,LocalizedObjectAndColors > position,1dsports_equipment 1esports_equipment 2dsports_equipment 2esports_equipment 3dsports_equipment 3esports_equipment > occur=and > textualMode=all > simReorder=false,{},NaN
346,179,179,vbs23-kis-v-m5,VISIONE,1,1673272765946,12656,51113.0,1780.0,2143.0,1780.0,9928.0,TEXT,LocalizedObjectAndColors > classes,sports_equipment > occur=and > textualMode=all > simReorder=false,{},NaN


### NOTES
- vitrivr-vr has a category called visualtextcoembedding, but only text information are preseted
- vitrivr-vr category and type columns are swapped
- vitrivr-vr additional temporal information '>'
- vitrivr-vr has only submittions for 16 of 19 tasks
- vitrivr value column is always empty
- vitrivr has a single sketch row
- vireo does not have category or type values 
- verge type=concept has empty or NaN value column
- verge has a single filter row
- HTW category=temporal has only text->text queries
- HTW should be renamed to vibro
- CVHunter has some prefixes in the query which were not typed by the user (CLIP: / Temporal CLIP:) gonna remove them in pre-processing
- CVHunter SUBMIT category does always have a video rank of INF
- CVHunter has several BROWSING types in the log, all have a video rank of INF
- CVHunter every query is send twice (with a 2-3s delay)
- VISIONE sends two queries if temporal, defined by "value":"...Temporal_query..." and the same timestamp 
- VISIONE additional information in the value column after the first '>'
- VISIONE in the value column "textualMode=all" means an ensemble of CLIP, Aladin, VideoClip

# Filtering and pre-processing
- remove category SUBMIT 
- remove VISIONE and HTW suffixes from "value"
- remove CVHunter prefixes from "value"
- remove some of CVHunter browsing types (those that does not alter ranking)
- swap content of column category and type for vitrivr-vr 
- remove vitrivr and vireo since they to not have any useable data

In [65]:
# remove category SUBMIT
dataset = dataset.loc[~((dataset["team"]=="CVHunter")&(dataset["category"]=="SUBMIT"))]

# remove category BROWSING
dataset = dataset.loc[~((dataset["team"]=="CVHunter")&(dataset["category"]=="BROWSING"))]

# remove text prefix for CVHunter queries
dataset.loc[((dataset["team"]=="CVHunter")&(dataset["category"]=="TEXT")),"value"] = dataset.loc[((dataset["team"]=="CVHunter")&(dataset["category"]=="TEXT")),"value"].str.replace("Temporal CLIP: ","")
dataset.loc[((dataset["team"]=="CVHunter")&(dataset["category"]=="TEXT")),"value"] = dataset.loc[((dataset["team"]=="CVHunter")&(dataset["category"]=="TEXT")),"value"].str.replace("CLIP: ","")

# rename HTW to vibro
dataset.loc[dataset["team"]=="HTW", "team"] = "vibro"

# remove vitrivr and VIREO for now because there are no text query information
dataset = dataset.loc[dataset["team"]!="vitrivr"]
dataset = dataset.loc[dataset["team"]!="VIREO"]

# remove concept queries
dataset = dataset.loc[dataset.type!="concept"]

# swap type and category column contents for vitrivr-VR
dataset.loc[dataset.team == "vitrivr-VR", ["type", "category"]] = dataset.loc[dataset.team == "vitrivr-VR", ["category", "type"]].values
dataset.shape

(995, 17)

### add temporal column and merge VISIONE's two row temporal queries

In [66]:
dataset["is_temporal_query"] = False

# mark the temporal queries
dataset.loc[((dataset["team"]=="vibro")&(dataset["category"]=="TEMPORAL")),"is_temporal_query"] = True
dataset.loc[((dataset["team"]=="CVHunter")&(dataset.value.str.contains('>'))),"is_temporal_query"] = True
dataset.loc[((dataset["team"]=="VISIONE")&(dataset.value.str.contains('Temporal_query'))),"is_temporal_query"] = True

# remove text suffix for VISIONE queries
dataset.loc[((dataset["team"]=="VISIONE")&(dataset["category"]=="TEXT")),"value"] = dataset.loc[((dataset["team"]=="VISIONE")&(dataset["category"]=="TEXT")),"value"].str.split('>').str[0]

# remove the ">" character
#dataset.loc[((dataset["team"]=="CVHunter")&(dataset["category"]=="TEXT")),"value"] = dataset.loc[((dataset["team"]=="CVHunter")&(dataset["category"]=="TEXT")),"value"].str.replace(">","")
#dataset.loc[((dataset["team"]=="vibro")&(dataset["category"]=="TEMPORAL")),"value"] = dataset.loc[((dataset["team"]=="vibro")&(dataset["category"]=="TEMPORAL")),"value"].str.replace(">","")

# concatenating temporal queries for VISIONE (query with same timestamp, user, and query type)
visione = dataset.loc[(dataset["team"]=="VISIONE")]
txtGroupby = visione.groupby(["task","team","user","timestamp","type","category"]).agg({'value': ' > '.join}).reset_index()
dataset_clean = visione.drop(columns=['value'])
dataset_visione = dataset_clean.merge(txtGroupby, on=["task","team","user","timestamp", "type", "category"], how='left')
dataset_visione = dataset_visione.drop_duplicates(subset=["task","team","user","timestamp","type","category"])
dataset_wo_visione = dataset.loc[(dataset["team"]!="VISIONE")]
dataset = pd.concat([dataset_wo_visione, dataset_visione])

print('\n'.join(map(str, dataset.loc[dataset.is_temporal_query][["team","category","type"]].groupby(["team","category","type"]).groups.keys())))

('CVHunter', 'TEXT', 'JointEmbedding')
('VISIONE', 'TEXT', 'jointEmbedding > ')
('vibro', 'TEMPORAL', 'text > text')


### add is joint embedding text query column

In [67]:
dataset["is_joint_embedding_text_query"] = True

dataset.loc[dataset.category=="IMAGE", "is_joint_embedding_text_query"] = False
dataset.loc[dataset.category=="FILTER", "is_joint_embedding_text_query"] = False
dataset.loc[dataset.type.str.contains("LocalizedObjectAndColors"), "is_joint_embedding_text_query"] = False

print('\n'.join(map(str, dataset.loc[dataset.is_joint_embedding_text_query][["team","category","type"]].groupby(["team","category","type"]).groups.keys())))

('CVHunter', 'TEXT', 'JointEmbedding')
('VISIONE', 'TEXT', 'jointEmbedding > ')
('Verge', 'TEXT', 'jointEmbedding')
('vibro', 'TEMPORAL', 'text > text')
('vibro', 'TEXT', 'jointEmbedding')
('vitrivr-VR', 'TEXT', 'mlclip')
('vitrivr-VR', 'TEXT', 'visualtextcoembedding')


### remove CVHunter query duplicates

In [68]:
dataset = dataset.sort_values(by='index')
dropped_rows = dataset[dataset.duplicated(subset=["task", "team", "user", "rank_video", "rank_shot_margin_0", "rank_shot_margin_5", "category", "type", "value"], keep='first')]
dropped_rows[["task", "team", "user", "value", "rank_video"]]

,task,team,user,value,rank_video
705,vbs23-kis-v1,CVHunter,1,"man in sport hall, backflip",inf
1486,vbs23-kis-v1,Verge,0,black guy doing backflip,inf
707,vbs23-kis-v1,CVHunter,0,man making salto on a stadion,inf
1488,vbs23-kis-v1,Verge,0,presenter with pink suit,inf
709,vbs23-kis-v1,CVHunter,1,"man in sport hall, backflip, american",inf
...,...,...,...,...,...
1302,vbs23-kis-v-m4,CVHunter,1,ocea bed with pillar corals and black fishes,3762.0
1303,vbs23-kis-v-m4,CVHunter,0,seabed with small corals and fish swarm,5027.0
1312,vbs23-kis-v-m4,CVHunter,1,ocea bed with pillar corals and black fishes in reddish water,4436.0
1355,vbs23-kis-v-m5,CVHunter,0,diver on his back over turqois sand,5.0


In [75]:
dataset_cvhunter = dataset.loc[(dataset["team"]=="CVHunter")&(dataset["category"]=="TEXT")].drop_duplicates(subset=["task", "team", "user", "category", "type", "value"], keep='first')
dataset_wo_cvhunter = dataset.loc[~((dataset["team"]=="CVHunter")&(dataset["category"]=="TEXT"))]
dataset = pd.concat([dataset_wo_cvhunter, dataset_cvhunter])
print(dataset.shape)

(784, 19)


### Sort table rows by a specific team order

In [73]:
dataset = dataset.sort_values(by='team', key=lambda x: x.map({v: i for i, v in enumerate(team_order)}))
dataset.team.unique()

array(['vibro', 'VISIONE', 'vitrivr-VR', 'CVHunter', 'Verge'],
      dtype=object)

### add CLIP feature column to every text query

In [76]:
import requests

def computeCLIP(text):
    
    url = 'https://navigu.net/feature/clipfv'
    data = {
        'text': text,
        'networks': 'CLIP-Textual'
    }

    response = requests.post(url, data=data)

    if response.status_code == 200:
        # Request was successful
        return np.frombuffer(response.content[-769:-1], dtype=np.int8)
    else:
        # There was an error
        print(f"Error: {response.status_code}")
        
# compute CLIP feature
dataset["joint_text_embedding"] = None
dataset.loc[dataset.is_joint_embedding_text_query, "joint_text_embedding"] = dataset.loc[dataset.is_joint_embedding_text_query, 'value'].apply(computeCLIP)
dataset["joint_text_embedding"][0]

0                  [123, 124, -125, -112, -124, -116, -121, 122, -109, 125, -117, -123, 124, -122, 126, -122, -127, 127, 121, -125, 122, -121, 124, 121, -116, 127, -126, -126, -126, 127, 124, -123, 123, -123, -126, 126, 118, 115, 126, -127, 125, 116, 127, 122, 125, -128, 124, -123, -128, -126, -123, 127, -115, -128, -119, 117, -125, 114, 125, -124, -128, -125, 112, 127, 125, 117, 121, -122, -120, -127, 123, 124, 124, 120, 123, 127, -123, 115, 121, 123, -127, -124, 111, 123, 123, -125, 124, 119, -126, 120, -128, -126, -125, 122, 119, 125, 125, 126, -124, -122, ...]
0    [-115, -109, -127, -125, -128, 121, -117, 124, 125, 126, -121, -114, -120, -113, -127, -126, -124, -117, -123, -127, -112, 118, 125, -123, 127, 126, 117, -128, -114, 116, -128, -126, -118, -125, -113, -122, -119, 118, 118, 114, -116, 121, 124, -122, -126, 124, -111, -120, 126, 121, 124, 117, -127, 121, 124, 122, 123, 120, -117, 117, 123, -119, 123, -119, -123, -124, -121, -126, 123, 119, -124, -116, -127, 122, -128, -126

# Create pickle file

In [77]:
dataset.to_pickle(comp_data["config"]["processed_logs_outdir"] + '/text_query_dataset.pkl')
print(dataset.shape)
print('\n'.join(map(str, dataset[["team","category","type"]].groupby(["team","category","type"]).groups.keys())))

(784, 20)
('CVHunter', 'IMAGE', 'JointEmbedding')
('CVHunter', 'IMAGE', 'feedbackModel')
('CVHunter', 'TEXT', 'JointEmbedding')
('VISIONE', 'IMAGE', 'globalFeatures > image similarity (GEM)')
('VISIONE', 'IMAGE', 'globalFeatures > sematic video similarity (CLIP)')
('VISIONE', 'TEXT', 'LocalizedObjectAndColors > classes')
('VISIONE', 'TEXT', 'LocalizedObjectAndColors > position')
('VISIONE', 'TEXT', 'jointEmbedding > ')
('Verge', 'FILTER', 'numberOfObjects')
('Verge', 'IMAGE', 'globalFeatures')
('Verge', 'TEXT', 'jointEmbedding')
('vibro', 'IMAGE', 'globalFeatures')
('vibro', 'TEMPORAL', 'text > text')
('vibro', 'TEXT', 'jointEmbedding')
('vitrivr-VR', 'TEXT', 'mlclip')
('vitrivr-VR', 'TEXT', 'visualtextcoembedding')
